In [1]:
import numpy as np
import pickle

In [2]:
from sklearn.metrics import confusion_matrix
def accuracy( C ):
    ''' Compute accuracy given Numpy array confusion matrix C. Returns a floating point value '''
    a = 0
    for i in range(4):
      a += C[i][i]
    
    return a/np.sum(C)

In [3]:
# get batches for each epoch
def get_batches(inputs, targets, batch_size, shuffle=True):
    """Divide a dataset into mini-batches of a given size. This is a'generator'."""
    
    if inputs.shape[0] % batch_size != 0:
        raise RuntimeError('The number of data points must be a multiple of the batch size.')
    num_batches = inputs.shape[0] // batch_size

    if shuffle:
        idxs = np.random.permutation(inputs.shape[0])
        inputs = inputs[idxs, :]
        targets = targets[idxs]

    for m in range(num_batches):
        yield inputs[m*batch_size:(m+1)*batch_size, :], \
              targets[m*batch_size:(m+1)*batch_size]

In [4]:
def train_test_split(inputs, targets, ratio=0.2, shuffle=True):
    num_train = int(inputs.shape[0]*(1-ratio))
    indices = np.arange(inputs.shape[0])
    if shuffle:
        np.random.shuffle(indices)
    X_train = inputs[indices[:num_train]]
    Y_train = targets[indices[:num_train]]
    X_test = inputs[indices[num_train:]]
    Y_test = targets[indices[num_train:]]
    return X_train, Y_train, X_test, Y_test

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.h1 = nn.Linear(164, 1058)
        self.o = nn.Linear(1058, 4)


    def forward(self, x):
        x = F.relu(self.h1(x))
        x = F.softmax(self.o(x))
        return x


net = Net()
print(net)
loss_f = F.cross_entropy




Net(
  (h1): Linear(in_features=164, out_features=1058, bias=True)
  (o): Linear(in_features=1058, out_features=4, bias=True)
)


In [6]:
# load data and labels
with open("dataSmaller.pk", "rb") as f:
    X, Y = pickle.load(f)

In [11]:
X_train, Y_train, X_test, Y_test = train_test_split(X, Y)

In [12]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((80000, 164), (80000,), (20000, 164), (20000,))

In [13]:
from torch.autograd import Variable
X_train = Variable(torch.from_numpy(X_train).float())
Y_train = Variable(torch.from_numpy(Y_train).long())
X_test = Variable(torch.from_numpy(X_test).float())
Y_test = Variable(torch.from_numpy(Y_test).long())

## train

In [ ]:
epoches = 100
lr = 0.0002
optimizer = optim.Adam(net.parameters(), lr=lr)
batch_size = 1000
loss_report = 10

for epoch in range(epoches):
    print("Epoch:", epoch+1)
    for i, (input_batch, target_batch) in enumerate(get_batches(X_train, Y_train, batch_size)):
        optimizer.zero_grad()   # zero the gradient buffers
        output = net(input_batch)
        loss = loss_f(output, target_batch)
        loss.backward()
        optimizer.step()    # Does the update
        if (i+1)%loss_report == 0:
            print("Batch:", i+1, "loss:",loss.item())
    # validation error
    with torch.no_grad():
        y_predict = net(X_test)
        loss_t = loss_f(y_predict, Y_test)
#         if loss_t < lbest:
#             lbest = loss_t
#             bestp = net.state_dict()
        y_p = torch.argmax(y_predict, dim = 1).numpy()
        c2 = confusion_matrix(Y_test, y_p)
        print("test loss:",loss_t.item(), "acc :", accuracy(c2))